In [2]:
#Primero vamos a importar todo lo necesario

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('regresion').getOrCreate()

#Para trabajar con ml y regresiones lineales tenemos que importar esto:
from pyspark.ml.regression import LinearRegression

In [3]:
#Importamos nuestros datos de clientes, en este caso vamos a importar un csv con la informacion
#usamos inferschema para que nos coja el tipo de dato y header true para que nos cree las cabeceras
datos = spark.read.csv("customers.csv",inferSchema = True,header = True)

In [4]:
#Vamos a ver los campos del csv
datos.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
#vemos que tenemos email, direccion, la duracion media de la sesion , tiempo en la aplicacion,tiempo en la web,
#el tiempo que lleva activa la cuenta, y cuanto ha gastado en el año

#si vemos los datos:
datos.show()# asi no se ve muy bien vemos mejor con head

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
datos.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [7]:
#vemos todos los datos, y el que vamos a querer predecir es el Yeaarly Amount Spent, que es lo que se gasta al año


#Debemos saber que para trabajar, el modelo va a necesiar un formato concreto que es de tipo vector que necesita spark
#para trbajar, esto lo vamos a conseguir 
#usando lo que se llama Assembler

#vamos a crear un Assembler el cual nos permite crear el formato que necesita el modelo para trabajar 
#con spark(formato de vector)
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
#vemos nuestras columnas:
datos.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
#y vamos a crear nuestro Assembler
#Crearemos con la funcion que acabamos de importar, añadimos en el inputCols las columnas que queremos que esten en nuestro vector:
#y el segundo elemento que es outputCols que se va a llamar features
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website','Length of Membership'],
                            outputCol='features')



In [10]:
#vamos a crear un output utilizando el assembles que acabamos de crear y transform usando los datos,para teenr los datos como
#necesitamos
output = assembler.transform(datos)

In [11]:
#si vemos los datos y ahcemos printschema vemos que tenemos una columna nueva llamada features
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
#vemos que se nos ha creado la colulmna features, y 
#si vemos lo que tiene esata columna debe de tener un vector con los datos que le hemos pasado: 
#los de media de la sesion, tiempo etc... los que vamos a usar en nuestra regresion lineal 
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [13]:
#vemos que ha creado el vector con los valores que hemos pasado y lo ha metido en la columna features

In [14]:
#vamos a coger los datos finales:
#que son el evctor que hemos creado que sera lo que nos ayude a predecir, y lo que queremos predecir que es el Yearly Amount Spent
datos_finales = output.select('features','Yearly Amount Spent')

In [15]:
datos_finales.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [16]:
#Entocnes a partir de estos datos hacemos un train y un test usando randomSplit, le vamos a dar 0,7 y 0,3
train, test = datos_finales.randomSplit([0.7,0.3])
#lo que estamos haciendo aqui es coger el 70% de los datos para entrenar el modelo y un 30 para testear el modelo

In [17]:
#vamos a ver train..
#vemos que tenemos 355 registros

train.describe().show()


+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean| 496.25968353126103|
| stddev|  78.68652306131044|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
#vemos que en test tenemos 145 registros
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean| 506.71980246047167|
| stddev|  80.60943345005087|
|    min| 327.37795258965207|
|    max|  725.5848140556806|
+-------+-------------------+



In [19]:
#Vamos a entrenar el modelo con train y evaluar el modelo con test

#Entonces vamos a crear una regresion lineal pasandole features que son los datos que vamos a usar para entrenar y
#El dato que queremos calcular en este caso el yearly amount Spent, y por ultimo añadiremos una columna de prediccion que
#sera el resultado que vamos a obtener

In [20]:

lr = LinearRegression(featuresCol = 'features',labelCol = 'Yearly Amount Spent',predictionCol = 'prediction')
#si por defento tienen los nombres features y label no hace falta indicarle el nombre, pero como aqui se 
#llama Yearly... se lo pasamos

In [21]:
lr_modelo = lr.fit(train) #Creamos el modelo con lr.fit() y le pasamos los datos de entrenamiento

In [22]:
resultado = lr_modelo.evaluate(test) #Evaluamos los datos test

In [23]:
resultado.residuals.show() #esto es lo que se ha equivocado por prediccion, el residual

+-------------------+
|          residuals|
+-------------------+
|  8.178019986211666|
|-11.662579206420503|
|0.28375387925063933|
|  23.88033922548516|
|  5.328554360332532|
| 1.5247878290938388|
| -4.585938823420406|
|  5.016282615349667|
|   17.7858835593359|
| -1.585812084553197|
|-3.2538229452102883|
|-16.324447019822003|
| 1.3243068169010712|
|  8.979596065303951|
| -16.35742247597409|
| -3.271236363257799|
|   4.72471901011221|
|  5.612109941364338|
| 7.1134779211603245|
|  23.82819269785142|
+-------------------+
only showing top 20 rows



In [24]:
resultado.rootMeanSquaredError #Es en lo que se ha equivocado, pro lo que si los precios son de 400 500 dolares, se esta
#equivocando en unos 9 dolares, por lo que es una muy buena prediccion
#Si el valor que tenemos que calcular es 9 y el error nos da 9, pues obviamente es muy mal resultado pero en neustro caso
#que son valores altos, 9 es muy poco

10.522935932789585

In [25]:
resultado.r2 #es un 0.98 que es muy bueno, siendo el maximo de acuierto 1, este nos esta dando un 0.98

0.9828411977757786

In [26]:
#Podemos hacer un sumary y ver las predicciones:
summary = lr_modelo.summary

In [27]:
summary.predictions.show()

+--------------------+-------------------+------------------+
|            features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.5324289670579...|  408.6403510726275| 396.2129552626909|
|[30.3931845423455...|  319.9288698031936| 330.7251777036697|
|[30.4925366965402...|  282.4712457199145| 287.1825725547285|
|[30.5743636841713...| 442.06441375806565| 440.3762661797134|
|[30.7377203726281...|  461.7807421962299|449.70867929914334|
|[30.8162006488763...|   266.086340948469| 282.6664667274133|
|[30.8364326747734...|  467.5019004269896|470.24814039939565|
|[30.8794843441274...|  490.2065999848547|492.21998777593853|
|[31.0472221394875...|  392.4973991890214|386.77926727686736|
|[31.0613251567161...|  487.5554580579016|  492.245354616555|
|[31.1239743499119...|  486.9470538397658|507.06124817285877|
|[31.1280900496166...|  557.2526867470547| 562.4946330032003|
|[31.1695067987115...|  427.3565308022928|416.07943078351013|
|[31.268

In [28]:
#Como vemos se acercan bastante al resultado original